<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
حالت سریع (Fast Mode): استفاده از ۲۰٪ داده‌ها
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
برخی مدل‌ها مانند <b>SVM</b> به‌خصوص با کرنل‌های غیرخطی (مثل RBF) زمان آموزش بالایی دارند.  
برای اینکه فرآیند تست و تنظیم پارامترها سریع‌تر انجام شود، در این نوتبوک فقط <b>۲۰٪</b> از داده‌های آموزشی را استفاده می‌کنیم.
<br><br>
برای اینکه نسبت کلاس‌ها به‌هم نریزد، نمونه‌گیری را به صورت <b>Stratified</b> انجام می‌دهیم تا توزیع <code>TripReason</code> در این ۲۰٪ مشابه کل داده باقی بماند.
</font>
</p>


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.metrics import f1_score, classification_report, confusion_matrix, make_scorer


In [ ]:
train_data = pd.read_csv("./data/train_processed.csv")
print("train:", train_data.shape)
train_data.head()


In [ ]:
X = train_data.drop(columns=["TripReason"])
y = train_data["TripReason"]
print("n_classes:", y.nunique())


In [ ]:
X_small, _, y_small, _ = train_test_split(
    X, y,
    train_size=0.2,          # فقط ۲۰٪
    random_state=42,
    stratify=y
)

print("small:", X_small.shape, y_small.shape)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42,
    stratify=y_small
)

print(X_train.shape, X_val.shape)


In [ ]:
f1_macro = make_scorer(f1_score, average="macro")
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)  # کمتر برای سرعت


In [ ]:
log_pipe = Pipeline(steps=[
    ("scaler", MinMaxScaler()),
    ("model", LogisticRegression(max_iter=5000))
])


In [ ]:
C_list = [0.01, 0.1, 1, 10, 30]
log_scores = []

for C in C_list:
    log_pipe.set_params(model__C=C, model__penalty="l2", model__solver="lbfgs")
    scores = cross_val_score(log_pipe, X_train, y_train, cv=cv, scoring=f1_macro)
    log_scores.append(scores.mean())
    print(f"[LOG] C={C:<5} => CV Macro-F1: {scores.mean():.4f} (+/- {scores.std():.4f})")

best_C = C_list[int(np.argmax(log_scores))]
print("\nBest Logistic C:", best_C)


In [ ]:
plt.figure(figsize=(7,4))
plt.plot(C_list, log_scores, marker="o")
plt.xscale("log")
plt.title("Logistic Regression: C vs CV Macro-F1 (Fast Mode)")
plt.xlabel("C (log scale)")
plt.ylabel("Macro-F1")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
final_log = Pipeline(steps=[
    ("scaler", MinMaxScaler()),
    ("model", LogisticRegression(C=best_C, penalty="l2", solver="lbfgs", max_iter=5000))
])

final_log.fit(X_train, y_train)
pred_log = final_log.predict(X_val)

print("Logistic Validation Macro-F1:", f1_score(y_val, pred_log, average="macro"))
print("\nConfusion Matrix:\n", confusion_matrix(y_val, pred_log))
print("\nClassification Report:\n", classification_report(y_val, pred_log))


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مدل‌سازی با SVM (Support Vector Machine)
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
SVM یکی از مدل‌های قدرتمند برای طبقه‌بندی است و با انتخاب کرنل مناسب می‌تواند مرزهای تصمیم پیچیده را یاد بگیرد.  
اما SVM معمولاً به مقیاس ویژگی‌ها حساس است؛ بنابراین برای عملکرد بهتر از <b>Scaling</b> استفاده می‌کنیم.
<br><br>
همچنین چون SVM می‌تواند زمان‌بر باشد، برای تست‌های اولیه از نمونه‌گیری ۲۰٪ و از Cross-Validation با تعداد fold کمتر استفاده می‌کنیم تا زمان اجرا منطقی بماند.
</font>
</p>


In [ ]:
svm_pipe = Pipeline(steps=[
    ("scaler", MinMaxScaler()),
    ("model", SVC())
])


In [ ]:
C_list = [0.1, 1, 10, 30]
svm_linear_results = []

for C in C_list:
    svm_pipe.set_params(model__kernel="linear", model__C=C)
    scores = cross_val_score(svm_pipe, X_train, y_train, cv=cv, scoring=f1_macro)
    svm_linear_results.append({
        "kernel": "linear",
        "C": C,
        "gamma": None,
        "cv_mean_f1": scores.mean(),
        "cv_std_f1": scores.std()
    })
    print(f"[SVM-linear] C={C:<5} => CV Macro-F1: {scores.mean():.4f} (+/- {scores.std():.4f})")


In [ ]:
C_list = [0.1, 1, 10]
gamma_list = ["scale", 0.1, 0.01]

svm_rbf_results = []

for C in C_list:
    for g in gamma_list:
        svm_pipe.set_params(model__kernel="rbf", model__C=C, model__gamma=g)
        scores = cross_val_score(svm_pipe, X_train, y_train, cv=cv, scoring=f1_macro)
        svm_rbf_results.append({
            "kernel": "rbf",
            "C": C,
            "gamma": g,
            "cv_mean_f1": scores.mean(),
            "cv_std_f1": scores.std()
        })
        print(f"[SVM-rbf] C={C:<4} gamma={str(g):<6} => CV Macro-F1: {scores.mean():.4f} (+/- {scores.std():.4f})")


In [ ]:
results_df = pd.DataFrame(svm_linear_results + svm_rbf_results).sort_values("cv_mean_f1", ascending=False)
results_df.head(10)


In [ ]:
best = results_df.iloc[0]
best_kernel = best["kernel"]
best_C = float(best["C"])
best_gamma = best["gamma"]

print("Best SVM params:", best.to_dict())


In [ ]:
# ساخت مدل نهایی با بهترین پارامترها
if best_kernel == "linear":
    final_svm = Pipeline(steps=[
        ("scaler", MinMaxScaler()),
        ("model", SVC(kernel="linear", C=best_C))
    ])
else:
    final_svm = Pipeline(steps=[
        ("scaler", MinMaxScaler()),
        ("model", SVC(kernel="rbf", C=best_C, gamma=best_gamma))
    ])

final_svm.fit(X_train, y_train)
pred_svm = final_svm.predict(X_val)

print("SVM Validation Macro-F1:", f1_score(y_val, pred_svm, average="macro"))
print("\nConfusion Matrix:\n", confusion_matrix(y_val, pred_svm))
print("\nClassification Report:\n", classification_report(y_val, pred_svm))
